In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# part 1

In [2]:
df = pd.read_csv('data/day7.csv',header=None)
df['command'] = df.apply(lambda s: True if '$' in s[0] else False, axis='columns')
df['cd'] = df.apply(lambda s: True if '$ cd' in s[0] else False, axis='columns')
df['cd_dir'] = df.apply(lambda s: s[0].replace("$ cd ","") if '$ cd' in s[0] else np.nan, axis='columns')
df['current_dir'] = df['cd_dir'].ffill()
df['dir'] = df.apply(lambda s: True if 'dir' in s[0] else False, axis='columns')
df['dir_name'] = df.apply(lambda s: s[0].replace("dir ","") if 'dir' in s[0] else "", axis='columns')
df['ls'] = df.apply(lambda s: True if '$ ls' in s[0] else False, axis='columns')
df['file_size'] = df.apply(lambda s: int(s[0].split(" ")[0]) if (not s['command']) and (not s['dir']) else 0, axis='columns')
df['file_name'] = df.apply(lambda s: s[0].split(" ")[1] if (not s['command']) and (not s['dir']) else "", axis='columns')

df.head(40)

,0,command,cd,cd_dir,current_dir,dir,dir_name,ls,file_size,file_name
0,$ cd /,True,True,/,/,False,,False,0,
1,$ ls,True,False,NaN,/,False,,True,0,
2,dir dpllhlcv,False,False,NaN,/,True,dpllhlcv,False,0,
3,284723 hznrlfhh.tnz,False,False,NaN,/,False,,False,284723,hznrlfhh.tnz
4,dir mgjdlmrz,False,False,NaN,/,True,mgjdlmrz,False,0,
5,dir njstc,False,False,NaN,/,True,njstc,False,0,
6,dir nzwbc,False,False,NaN,/,True,nzwbc,False,0,
7,dir qzzfvdh,False,False,NaN,/,True,qzzfvdh,False,0,
8,dir smvhphf,False,False,NaN,/,True,smvhphf,False,0,
9,$ cd dpllhlcv,True,True,dpllhlcv,dpllhlcv,False,,False,0,


In [17]:
# plan
# construct the directory structure
# find the number of files at each directory level
# aggregate up the number of files in the previous levels

current_dir = None
agg_file_size = 0
outs = []

for index,s in df.iterrows():
    if s['cd']:
        if current_dir is None: # if its the first time the function is hitting here
            current_dir = Path(s['cd_dir'])
            continue
        
        # whether we're navigating up or down, save the data found and start new directory with zero file size
        if previous_command['cd_dir'] != "..":
            outs.append((current_dir, agg_file_size))
        if s['cd_dir'] == '..':
            current_dir = current_dir.parent
        else:
            current_dir = Path(current_dir, s['cd_dir']) # start a new directory listing
        agg_file_size = 0 # reset the file size counter

    elif not s['command']: # if its not a command its a file or a directory
        if not s['dir']: #its a file
            agg_file_size += s['file_size']
    previous_command = s
    
outs.append((current_dir, agg_file_size)) #to commit last file to outs

outs = pd.DataFrame.from_records(outs).rename(columns={0:'dir',1:'file'}).set_index('dir')['file']
outs.index = [f'/{str(i)[1:].replace("/", "_")}' for i in outs.index]
outs#.head(20)

/                                  284723
/dpllhlcv                           11223
/dpllhlcv_gpmlznd                  178313
/dpllhlcv_gpmlznd_lwzcss                0
/dpllhlcv_gpmlznd_lwzcss_bttplh         0
                                    ...  
/qzzfvdh_vjqzf_gnnstzc             181151
/qzzfvdh_zhmgmmv                   165656
/qzzfvdh_zhmgmmv_fvwm                   0
/qzzfvdh_zhmgmmv_fvwm_dgpfcftj     214069
/smvhphf                           202817
Name: file, Length: 183, dtype: int64

In [7]:
finalouts = outs.copy()
for i,v in finalouts.items():
    size = 0
    for j,k in outs.items():
        if i in j:
            size += k
    finalouts[i] = size
finalouts.head(20)

/                                           41735494
/dpllhlcv                                    3339835
/dpllhlcv_gpmlznd                             279445
/dpllhlcv_gpmlznd_lwzcss                       59866
/dpllhlcv_gpmlznd_lwzcss_bttplh                59866
/dpllhlcv_gpmlznd_lwzcss_bttplh_rzj            59866
/dpllhlcv_gpmlznd_rhdllvm                      41266
/dpllhlcv_gpmlznd_rhdllvm_mvqfrq               41266
/dpllhlcv_pgcctrb                             101757
/dpllhlcv_pgcctrb_dgpfcftj                    101757
/dpllhlcv_wmsl                               2947410
/dpllhlcv_wmsl_hhtztpm                       1134787
/dpllhlcv_wmsl_hhtztpm_ngjd                   289546
/dpllhlcv_wmsl_hhtztpm_wmsl                   178765
/dpllhlcv_wmsl_tzjthwc                       1399822
/dpllhlcv_wmsl_tzjthwc_lhccf                  712782
/dpllhlcv_wmsl_tzjthwc_lhccf_nzwbc            102053
/dpllhlcv_wmsl_tzjthwc_lhccf_nzwbc_wmsl       102053
/dpllhlcv_wmsl_tzjthwc_lhccf_zfwffjn          

In [9]:
finalouts[finalouts<=100_000].sum()

1648397

# part 2

In [19]:
space_that_needs_freeing = finalouts['/'] - 40_000_000
space_that_needs_freeing

1735494

In [23]:
finalouts[finalouts<=(space_that_needs_freeing*1.05)].sort_values(ascending=False)

/nzwbc_zcv_nbq                             1815525
/mgjdlmrz_fnfw_dhsvtfc_cwbq_ccgdn          1597017
/mgjdlmrz_fnfw_bhjm_rrrj                   1546977
/mgjdlmrz_fnfw_bhjm_bpl                    1474657
/dpllhlcv_wmsl_tzjthwc                     1399822
                                            ...   
/mgjdlmrz_fnfw_bhjm_rrrj_dgpfcftj            27856
/nzwbc_mfbhtb                                21538
/mgjdlmrz_nfqzjs                             18103
/qzzfvdh_tzjthwc_snjv_grq_nzwbc              12818
/mgjdlmrz_fnfw_bhjm_rrrj_tnpjsgnq_gbddb       1587
Name: file, Length: 162, dtype: int64